In [3]:
# Imports principales para el forecasting
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import streamlit as st
import holidays
import joblib

In [6]:
# ============================================
# CARGA DE MODELO Y DATASETS
# ============================================

# 1️⃣ Cargar df_merged del entrenamiento
df_merged = pd.read_csv("../data/processed/df_merged.csv")
df_merged['fecha'] = pd.to_datetime(df_merged['fecha'])

print("✔️ df_merged cargado:", df_merged.shape)

# 2️⃣ Reconstruir columnas del modelo final (predictoras)
columnas_excluir = ['fecha', 'ingresos', 'unidades_vendidas']
columnas_excluir = [col for col in columnas_excluir if col in df_merged.columns]

predictoras = df_merged.select_dtypes(exclude=['object', 'datetime']).drop(
    columns=columnas_excluir,
    errors='ignore'
).columns.tolist()

columnas_modelo = predictoras

print("✔️ Columnas del modelo final reconstruidas:", len(columnas_modelo))

# 3️⃣ Cargar el modelo final
model_final = joblib.load("../models/modelo_final.joblib")

print("✔️ Modelo final cargado.")


✔️ df_merged cargado: (3524, 79)
✔️ Columnas del modelo final reconstruidas: 69
✔️ Modelo final cargado.


In [14]:
# =====================================================
# TRANSFORMACIÓN COMPLETA DEL DF_INFERENCIA PARA PREDECIR
# =====================================================

# 1️⃣ Cargar df_inferencia
df_inf = pd.read_csv("../data/raw/inferencia/ventas_2025_inferencia.csv")

# 2️⃣ Asegurar formato datetime
df_inf['fecha'] = pd.to_datetime(df_inf['fecha'])

# -----------------------------------------------------
# 3️⃣ Variables temporales
# -----------------------------------------------------
df_inf["anio"] = df_inf["fecha"].dt.year
df_inf["mes"] = df_inf["fecha"].dt.month
df_inf["dia_mes"] = df_inf["fecha"].dt.day
df_inf["dia_semana"] = df_inf["fecha"].dt.dayofweek
df_inf["es_fin_de_semana"] = df_inf["dia_semana"].isin([5, 6]).astype(int)
df_inf["semana_anio"] = df_inf["fecha"].dt.isocalendar().week

trimestre_map = {1:1,2:1,3:1,4:2,5:2,6:2,7:3,8:3,9:3,10:4,11:4,12:4}
df_inf["trimestre"] = df_inf["mes"].map(trimestre_map)

# -----------------------------------------------------
# 4️⃣ Variables económicas
# -----------------------------------------------------
df_inf["ratio_precio"] = df_inf["precio_venta"] / df_inf["precio_base"]
df_inf["descuento_porcentaje"] = (
    (df_inf["precio_base"] - df_inf["precio_venta"]) / df_inf["precio_base"]
) * 100

# -----------------------------------------------------
# 5️⃣ One-hot encoding
# -----------------------------------------------------
df_inf = pd.get_dummies(
    df_inf,
    columns=["categoria", "subcategoria", "nombre"],
    prefix=["categoria_h", "subcategoria_h", "nombre_h"],
)

# -----------------------------------------------------
# 6️⃣ Crear lags y media móvil usando histórico REAL
# -----------------------------------------------------
df_tmp = pd.concat(
    [
        df_merged[["fecha", "producto_id", "unidades_vendidas"]],
        df_inf[["fecha", "producto_id"]],
    ],
    ignore_index=True,
)

df_tmp = df_tmp.sort_values(["producto_id", "fecha"])

# Crear lags
for lag in range(1, 8):
    df_tmp[f"lag_{lag}"] = df_tmp.groupby("producto_id")["unidades_vendidas"].shift(lag)

# Media móvil
df_tmp["media_movil_7"] = (
    df_tmp.groupby("producto_id")["unidades_vendidas"]
    .rolling(7)
    .mean()
    .reset_index(0, drop=True)
)

df_inf = df_inf.merge(
    df_tmp[
        ["fecha", "producto_id"]
        + [f"lag_{i}" for i in range(1, 8)]
        + ["media_movil_7"]
    ],
    on=["fecha", "producto_id"],
    how="left",
)

# -----------------------------------------------------
# 7️⃣ Guardar fecha para filtrar
# -----------------------------------------------------
df_inf["fecha_backup"] = df_inf["fecha"]

# -----------------------------------------------------
# 8️⃣ FILTRAR SOLO noviembre 2025 (AHORA SÍ)
# -----------------------------------------------------
df_inf = df_inf[
    (df_inf["fecha_backup"] >= "2025-11-01")
    & (df_inf["fecha_backup"] <= "2025-11-30")
]

print("Después de filtrar noviembre:", df_inf.shape)

# -----------------------------------------------------
# 9️⃣ Alinear columnas del modelo
# -----------------------------------------------------
columnas_excluir = ["fecha", "ingresos", "unidades_vendidas"]
columnas_excluir = [c for c in columnas_excluir if c in df_merged.columns]

columnas_modelo = (
    df_merged.select_dtypes(exclude=["object", "datetime"])
    .drop(columns=columnas_excluir, errors="ignore")
    .columns.tolist()
)

# Crear columnas faltantes
for col in columnas_modelo:
    if col not in df_inf.columns:
        df_inf[col] = 0

df_inf = df_inf[columnas_modelo]

print("✔️ df_inferencia preparado correctamente.")


Después de filtrar noviembre: (720, 72)
✔️ df_inferencia preparado correctamente.


In [15]:
# Alinear columnas con el modelo final (quedarán 69 columnas)
df_inf = df_inf[columnas_modelo]

# Generar predicciones
df_inf["predicciones"] = model_final.predict(df_inf)

print("✔️ Predicciones generadas correctamente")
print("Shape final:", df_inf.shape)

✔️ Predicciones generadas correctamente
Shape final: (720, 70)


In [16]:
# Cargar el raw original para recuperar fecha y producto
df_raw = pd.read_csv("../data/raw/inferencia/ventas_2025_inferencia.csv")
df_raw["fecha"] = pd.to_datetime(df_raw["fecha"])

# Filtrar noviembre igual que en df_inf
df_raw = df_raw[
    (df_raw["fecha"] >= "2025-11-01") &
    (df_raw["fecha"] <= "2025-11-30")
]

# Reset index para alinear con df_inf
df_raw = df_raw.reset_index(drop=True)

# Construir dataframe final
df_pred_final = pd.DataFrame({
    "fecha": df_raw["fecha"],
    "producto_id": df_raw["producto_id"],
    "nombre": df_raw["nombre"],
    "categoria": df_raw["categoria"],
    "subcategoria": df_raw["subcategoria"],
    "precio_venta": df_raw["precio_venta"],
    "precio_base": df_raw["precio_base"],
    "prediccion": df_inf["predicciones"]
})

df_pred_final.head()

,fecha,producto_id,nombre,categoria,subcategoria,precio_venta,precio_base,prediccion
0,2025-11-01,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115.00,115.0,NaN
1,2025-11-01,PROD_002,Adidas Ultraboost 23,Running,Zapatillas Running,135.00,135.0,NaN
2,2025-11-01,PROD_003,Asics Gel Nimbus 25,Running,Zapatillas Running,86.39,85.0,NaN
3,2025-11-01,PROD_004,New Balance Fresh Foam X 1080v12,Running,Zapatillas Running,74.09,75.0,NaN
4,2025-11-01,PROD_005,Nike Dri-FIT Miler,Running,Ropa Running,34.76,35.0,NaN


In [17]:
output_path = "../data/processed/predicciones_noviembre_2025.csv"
df_pred_final.to_csv(output_path, index=False)

print("✔️ Archivo guardado en:", output_path)

✔️ Archivo guardado en: ../data/processed/predicciones_noviembre_2025.csv


In [18]:
set(df_inf.columns) - set(columnas_modelo)

{'predicciones'}

In [19]:
# =====================================================
# REGENERAR df_inf COMPLETO DESDE CERO
# =====================================================

# 1️⃣ Cargar df original
df_inf = pd.read_csv("../data/raw/inferencia/ventas_2025_inferencia.csv")
df_inf["fecha"] = pd.to_datetime(df_inf["fecha"])

# 2️⃣ Variables temporales
df_inf["anio"] = df_inf["fecha"].dt.year
df_inf["mes"] = df_inf["fecha"].dt.month
df_inf["dia_mes"] = df_inf["fecha"].dt.day
df_inf["dia_semana"] = df_inf["fecha"].dt.dayofweek
df_inf["es_fin_de_semana"] = df_inf["dia_semana"].isin([5,6]).astype(int)

# 3️⃣ Económicas
df_inf["ratio_precio"] = df_inf["precio_venta"] / df_inf["precio_base"]
df_inf["descuento_porcentaje"] = (
    (df_inf["precio_base"] - df_inf["precio_venta"]) / df_inf["precio_base"]
) * 100

# 4️⃣ One-hot encoding
df_inf = pd.get_dummies(
    df_inf,
    columns=["categoria","subcategoria","nombre"],
    prefix=["categoria_h","subcategoria_h","nombre_h"]
)

# 5️⃣ Crear lags usando df_merged + inferencia
df_tmp = pd.concat(
    [
        df_merged[["fecha","producto_id","unidades_vendidas"]],
        df_inf[["fecha","producto_id"]],
    ],
    ignore_index=True,
)

df_tmp = df_tmp.sort_values(["producto_id","fecha"])

# lags
for lag in range(1,8):
    df_tmp[f"lag_{lag}"] = df_tmp.groupby("producto_id")["unidades_vendidas"].shift(lag)

# media móvil
df_tmp["media_movil_7"] = (
    df_tmp.groupby("producto_id")["unidades_vendidas"]
    .rolling(7).mean()
    .reset_index(0,drop=True)
)

df_inf = df_inf.merge(
    df_tmp[
        ["fecha","producto_id"]
        + [f"lag_{i}" for i in range(1,8)]
        + ["media_movil_7"]
    ],
    on=["fecha","producto_id"],
    how="left"
)

# 6️⃣ Filtrar noviembre 2025 (fecha válida)
df_inf = df_inf[
    (df_inf["fecha"] >= "2025-11-01") &
    (df_inf["fecha"] <= "2025-11-30")
]

print("Después de filtrar:", df_inf.shape)

# 7️⃣ Alinear con columnas_modelo
# Rellenar las faltantes
for col in columnas_modelo:
    if col not in df_inf.columns:
        df_inf[col] = 0

# Eliminar extras
for col in df_inf.columns:
    if col not in columnas_modelo:
        df_inf = df_inf.drop(columns=[col])

# Reordenar
df_inf = df_inf[columnas_modelo]

print("Shape alineado:", df_inf.shape)

# 8️⃣ Generar predicciones
df_inf["predicciones"] = model_final.predict(df_inf)

print("✔️ Predicciones generadas.")
df_inf.head()


Después de filtrar: (720, 69)
Shape alineado: (720, 69)
✔️ Predicciones generadas.


,precio_base,es_estrella,precio_venta,año,anio,mes,dia_mes,semana_anio,trimestre,es_fin_de_semana,...,subcategoria_h_Mancuernas Ajustables,subcategoria_h_Mochila Trekking,subcategoria_h_Pesa Rusa,subcategoria_h_Pesas Casa,subcategoria_h_Rodillera Yoga,subcategoria_h_Ropa Montaña,subcategoria_h_Ropa Running,subcategoria_h_Zapatillas Running,subcategoria_h_Zapatillas Trail,predicciones
168,115,True,115.00,0,2025,11,1,0,0,1,...,False,False,False,False,False,False,False,True,False,12.253826
169,135,True,135.00,0,2025,11,1,0,0,1,...,False,False,False,False,False,False,False,True,False,9.939096
170,85,False,86.39,0,2025,11,1,0,0,1,...,False,False,False,False,False,False,False,True,False,3.009926
171,75,False,74.09,0,2025,11,1,0,0,1,...,False,False,False,False,False,False,False,True,False,3.009926
172,35,False,34.76,0,2025,11,1,0,0,1,...,False,False,False,False,False,False,True,False,False,3.079564


In [22]:
df_raw = pd.read_csv("../data/raw/inferencia/ventas_2025_inferencia.csv")
df_raw["fecha"] = pd.to_datetime(df_raw["fecha"])

# Filtrar noviembre 2025
df_raw = df_raw[
    (df_raw["fecha"] >= "2025-11-01") &
    (df_raw["fecha"] <= "2025-11-30")
].reset_index(drop=True)

# Crear dataframe final alineado por posición
df_pred_final = df_raw.copy()
df_pred_final["prediccion"] = df_inf["predicciones"].values

df_pred_final.head()

,fecha,producto_id,nombre,categoria,subcategoria,precio_base,es_estrella,unidades_vendidas,precio_venta,ingresos,Amazon,Decathlon,Deporvillage,prediccion
0,2025-11-01,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,NaN,115.00,NaN,81.78,117.08,102.97,12.253826
1,2025-11-01,PROD_002,Adidas Ultraboost 23,Running,Zapatillas Running,135,True,NaN,135.00,NaN,121.00,116.13,120.11,9.939096
2,2025-11-01,PROD_003,Asics Gel Nimbus 25,Running,Zapatillas Running,85,False,NaN,86.39,NaN,88.64,74.21,85.22,3.009926
3,2025-11-01,PROD_004,New Balance Fresh Foam X 1080v12,Running,Zapatillas Running,75,False,NaN,74.09,NaN,85.42,65.98,76.98,3.009926
4,2025-11-01,PROD_005,Nike Dri-FIT Miler,Running,Ropa Running,35,False,NaN,34.76,NaN,36.95,32.91,35.69,3.079564


In [23]:
df_entrega = df_pred_final.drop(columns=["unidades_vendidas", "ingresos"])

df_entrega.to_csv("../data/processed/predicciones_noviembre_2025.csv", index=False)

df_entrega.head()

,fecha,producto_id,nombre,categoria,subcategoria,precio_base,es_estrella,precio_venta,Amazon,Decathlon,Deporvillage,prediccion
0,2025-11-01,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,115.00,81.78,117.08,102.97,12.253826
1,2025-11-01,PROD_002,Adidas Ultraboost 23,Running,Zapatillas Running,135,True,135.00,121.00,116.13,120.11,9.939096
2,2025-11-01,PROD_003,Asics Gel Nimbus 25,Running,Zapatillas Running,85,False,86.39,88.64,74.21,85.22,3.009926
3,2025-11-01,PROD_004,New Balance Fresh Foam X 1080v12,Running,Zapatillas Running,75,False,74.09,85.42,65.98,76.98,3.009926
4,2025-11-01,PROD_005,Nike Dri-FIT Miler,Running,Ropa Running,35,False,34.76,36.95,32.91,35.69,3.079564


In [25]:
output_path = "../data/processed/df_inferencia_transformado.csv"
df_inf.to_csv(output_path, index=False)

print("✔️ Archivo guardado en:", output_path)

✔️ Archivo guardado en: ../data/processed/df_inferencia_transformado.csv
